## SeeClickFix2

The previous iteration works well but there are some missing features such as geo location. Also airflow was ran on my local computer, which is a no-no because I have raspberrypi's and a dedicated server.
Changes:
1. installed postgresql on raspberrypi 4
2. delete postgresql from opitplex
3. install airflow on optiplex
4. recreate dashboard on kibana but with geo-location capabilities this time. this require the correct format of data

    Exploratory data analysis
Data source: see click fix api
Targeted data topics:
    request_id  bigint
    description text (has to be normalized)
    zipcode     int
    latitude    double precision
    longitude   double precision
    address     text
    date        date

lat and long is going to be converted to "coordinate" with format "coordinate": {  "lat": -6.2008, "lon": 106.8456}

rename address to full address. extract street and city out of address

In [33]:
import requests
import json
import pandas as pd
import os

In [53]:
url = 'https://seeclickfix.com/api/v2/issues?place_url=santa-clara-county'
try:
    r = requests.get(url).content
except:
    print("Trouble fetch http request from seeclickfix")
data = json.loads(r.decode('utf-8'))
df = pd.DataFrame(data['issues']).reset_index()
#df.to_csv('issues.csv')
df.head(3)

,index,id,status,summary,description,rating,lat,lng,address,created_at,...,url,point,private_visibility,html_url,request_type,comment_url,flag_url,transitions,reporter,media
0,0,13718508,Acknowledged,Park Lighting,Black light pole is loose and wobbles side to ...,2,37.438771,-121.883577,"Milpitas CA, USA",2022-11-19T13:55:23-05:00,...,https://seeclickfix.com/api/v2/issues/13718508,"{'type': 'Point', 'coordinates': [-121.8835773...",False,https://seeclickfix.com/issues/13718508,"{'id': 42195, 'title': 'Park Lighting', 'organ...",https://seeclickfix.com/api/v2/issues/13718508...,https://seeclickfix.com/api/v2/issues/13718508...,{},"{'id': 1720984, 'name': 'City of Milpitas Neig...","{'video_url': None, 'image_full': 'https://see..."
1,1,13718464,Acknowledged,Graffiti Removal,graffiti 16315 condit south and east sides of ...,2,37.124252,-121.626975,"16301-16483 Condit Rd Morgan Hill, CA, 95037, USA",2022-11-19T13:39:47-05:00,...,https://seeclickfix.com/api/v2/issues/13718464,"{'type': 'Point', 'coordinates': [-121.6269747...",False,https://seeclickfix.com/issues/13718464,"{'id': 38354, 'title': 'Graffiti Removal', 'or...",https://seeclickfix.com/api/v2/issues/13718464...,https://seeclickfix.com/api/v2/issues/13718464...,{},"{'id': 2622990, 'name': 'DougM', 'role': 'Regi...","{'video_url': None, 'image_full': 'https://see..."
2,2,13717668,Acknowledged,Vandalism / Graffitti,My vehicle was damaged by a premediated group ...,3,37.428398,-121.919815,"190 Ranch Dr Milpitas, California, 95035",2022-11-19T00:50:18-05:00,...,https://seeclickfix.com/api/v2/issues/13717668,"{'type': 'Point', 'coordinates': [-121.9198150...",False,https://seeclickfix.com/issues/13717668,"{'id': 33151, 'title': 'Vandalism / Graffitti'...",https://seeclickfix.com/api/v2/issues/13717668...,https://seeclickfix.com/api/v2/issues/13717668...,{},"{'id': 1720984, 'name': 'City of Milpitas Neig...","{'video_url': None, 'image_full': None, 'image..."


    Splitting datetime into date and time - dropping timezone since it's located in one area

In [54]:
datetime = df['created_at'].str.split(pat='T',n=1,expand=True)
date = datetime[0]
time = datetime[1].str.split(pat='-',expand=True,n=1)
time = time[0]

In [55]:
df['date']=date[0]
df['time']=time[0]

    Rename address into full_address

In [56]:
df = df.rename(columns={'address':'full_address'})

In [57]:
df.head(1)

,index,id,status,summary,description,rating,lat,lng,full_address,created_at,...,private_visibility,html_url,request_type,comment_url,flag_url,transitions,reporter,media,date,time
0,0,13718508,Acknowledged,Park Lighting,Black light pole is loose and wobbles side to ...,2,37.438771,-121.883577,"Milpitas CA, USA",2022-11-19T13:55:23-05:00,...,False,https://seeclickfix.com/issues/13718508,"{'id': 42195, 'title': 'Park Lighting', 'organ...",https://seeclickfix.com/api/v2/issues/13718508...,https://seeclickfix.com/api/v2/issues/13718508...,{},"{'id': 1720984, 'name': 'City of Milpitas Neig...","{'video_url': None, 'image_full': 'https://see...",2022-11-19,13:55:23


    Let's extract zip code from full address using regex

In [62]:
street = df['full_address']
df['zip_code'] = street.str.extract(r'(9\d{4})')

In [64]:
df['zip_code'].fillna(value=95051,inplace=True)

In [65]:
df

,index,id,status,summary,description,rating,lat,lng,full_address,created_at,...,html_url,request_type,comment_url,flag_url,transitions,reporter,media,date,time,zip_code
0,0,13718508,Acknowledged,Park Lighting,Black light pole is loose and wobbles side to ...,2,37.438771,-121.883577,"Milpitas CA, USA",2022-11-19T13:55:23-05:00,...,https://seeclickfix.com/issues/13718508,"{'id': 42195, 'title': 'Park Lighting', 'organ...",https://seeclickfix.com/api/v2/issues/13718508...,https://seeclickfix.com/api/v2/issues/13718508...,{},"{'id': 1720984, 'name': 'City of Milpitas Neig...","{'video_url': None, 'image_full': 'https://see...",2022-11-19,13:55:23,95051
1,1,13718464,Acknowledged,Graffiti Removal,graffiti 16315 condit south and east sides of ...,2,37.124252,-121.626975,"16301-16483 Condit Rd Morgan Hill, CA, 95037, USA",2022-11-19T13:39:47-05:00,...,https://seeclickfix.com/issues/13718464,"{'id': 38354, 'title': 'Graffiti Removal', 'or...",https://seeclickfix.com/api/v2/issues/13718464...,https://seeclickfix.com/api/v2/issues/13718464...,{},"{'id': 2622990, 'name': 'DougM', 'role': 'Regi...","{'video_url': None, 'image_full': 'https://see...",2022-11-19,13:55:23,95037
2,2,13717668,Acknowledged,Vandalism / Graffitti,My vehicle was damaged by a premediated group ...,3,37.428398,-121.919815,"190 Ranch Dr Milpitas, California, 95035",2022-11-19T00:50:18-05:00,...,https://seeclickfix.com/issues/13717668,"{'id': 33151, 'title': 'Vandalism / Graffitti'...",https://seeclickfix.com/api/v2/issues/13717668...,https://seeclickfix.com/api/v2/issues/13717668...,{},"{'id': 1720984, 'name': 'City of Milpitas Neig...","{'video_url': None, 'image_full': None, 'image...",2022-11-19,13:55:23,95035
3,3,13717642,Acknowledged,FM - Indoor Construction,Padding coming off this basketball hoop in eas...,2,37.113241,-121.645780,"171 W Edmundson Ave Morgan Hill CA 95037, Unit...",2022-11-19T00:19:06-05:00,...,https://seeclickfix.com/issues/13717642,"{'id': 38079, 'title': 'FM - Indoor Constructi...",https://seeclickfix.com/api/v2/issues/13717642...,https://seeclickfix.com/api/v2/issues/13717642...,{},"{'id': 2568734, 'name': 'MOD', 'role': 'Verifi...","{'video_url': None, 'image_full': 'https://see...",2022-11-19,13:55:23,95037
4,4,13717574,Open,Traffic or Street Sign Issue,Parents frequently are ignore the sign for li...,2,37.239288,-121.981868,"17785 Vista Ave Monte Sereno, CA, 95030, USA",2022-11-18T22:28:38-05:00,...,https://seeclickfix.com/issues/13717574,"{'id': 34709, 'title': 'Traffic or Street Sign...",https://seeclickfix.com/api/v2/issues/13717574...,https://seeclickfix.com/api/v2/issues/13717574...,{'close_url': 'https://seeclickfix.com/api/v2/...,"{'id': 2662288, 'name': 'Philip Levine', 'role...","{'video_url': None, 'image_full': None, 'image...",2022-11-19,13:55:23,95030
5,5,13717564,Acknowledged,Sidewalk Repair,Sidewalk tripping hazard. Looks like prior rep...,2,37.430077,-121.875830,"1571 Edsel Dr Milpitas CA 95035, United States",2022-11-18T22:12:56-05:00,...,https://seeclickfix.com/issues/13717564,"{'id': 26656, 'title': 'Sidewalk Repair', 'org...",https://seeclickfix.com/api/v2/issues/13717564...,https://seeclickfix.com/api/v2/issues/13717564...,{},"{'id': 0, 'name': 'An anonymous SeeClickFix us...","{'video_url': None, 'image_full': 'https://see...",2022-11-19,13:55:23,95035
6,6,13717556,Open,Sidewalk Issues,,1,37.239118,-121.977319,"17509 Via Sereno Monte Sereno, CA, 95030, USA",2022-11-18T22:05:27-05:00,...,https://seeclickfix.com/issues/13717556,"{'id': 34706, 'title': 'Sidewalk Issues', 'org...",https://seeclickfix.com/api/v2/issues/13717556...,https://seeclickfix.com/api/v2/issues/13717556...,{'close_url': 'https://seeclickfix.com/api/v2/...,"{'id': 2662288, 'name': 'Philip Levine', 'role...","{'video_url': None, 'image_full': None, 'image...",2022-11-19,13:55:23,95030
7,7,13717555,Open,Sidewalk Issues,,1,37.239118,-121.977319,"17509 Via Sereno Monte Sereno, CA, 95030, USA",2022-11-18T22:05:00-05:00,...,https://seeclickfix.com/issues/1

    Convert long at into elastic geopoints

In [72]:
!pip install pygeohash

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py install for pygeohash: started
  Running setup.py install for pygeohash: finished with status 'done'


  DEPRECATION: pygeohash is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559


In [73]:
import pygeohash as gh

In [71]:
longlat = df[['lat','lng']]
longlat['coordinate'] = '{"lat":' + longlat['lat'].astype(str) + ',' + '"lon":' + longlat['lng'].astype(str) + '}'
longlat.head(3)

C:\Users\antng\AppData\Local\Temp\ipykernel_11164\738672038.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  longlat['coordinate'] = '{"lat":' + longlat['lat'].astype(str) + ',' + '"lon":' + longlat['lng'].astype(str) + '}'


,lat,lng,coordinate
0,37.438771,-121.883577,"{""lat"":37.4387707321218,""lon"":-121.883577365609}"
1,37.124252,-121.626975,"{""lat"":37.1242523729142,""lon"":-121.626974723278}"
2,37.428398,-121.919815,"{""lat"":37.4283980213903,""lon"":-121.919815019667}"


In [82]:
longlat['geohash']=longlat.apply(lambda x: gh.encode(x.lat, x.lng, precision=10), axis=1)

C:\Users\antng\AppData\Local\Temp\ipykernel_11164\432868116.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  longlat['geohash']=longlat.apply(lambda x: gh.encode(x.lat, x.lng, precision=10), axis=1)


In [83]:
longlat.head(1)

,lat,lng,coordinate,geohash
0,37.438771,-121.883577,"{""lat"":37.4387707321218,""lon"":-121.883577365609}",9q9kfrx0du


In [84]:
df['coordinate']=longlat['geohash']

    Now, we collect all the required fields into one dataframe

In [43]:
our_df = df[['id','summary','description','rating','full_address','html_url','date','time','zip_code','coordinate']]

In [44]:
our_df.head(3)

,id,summary,description,rating,full_address,html_url,date,time,zip_code,coordinate
0,13718508,Park Lighting,Black light pole is loose and wobbles side to ...,2,"Milpitas CA, USA",https://seeclickfix.com/issues/13718508,2022-11-19,13:55:23,NaN,"{""lat"":37.4387707321218,""lon"":-121.883577365609}"
1,13718464,Graffiti Removal,graffiti 16315 condit south and east sides of ...,2,"16301-16483 Condit Rd Morgan Hill, CA, 95037, USA",https://seeclickfix.com/issues/13718464,2022-11-19,13:55:23,16301,"{""lat"":37.1242523729142,""lon"":-121.626974723278}"
2,13717668,Vandalism / Graffitti,My vehicle was damaged by a premediated group ...,3,"190 Ranch Dr Milpitas, California, 95035",https://seeclickfix.com/issues/13717668,2022-11-19,13:55:23,95035,"{""lat"":37.4283980213903,""lon"":-121.919815019667}"


In [45]:
our_df.to_csv('filtered.csv')

    Lovely :) our data is now somewhat clean

In [46]:
import datetime as dt
from datetime import timedelta
import psycopg2 as db

In [51]:
con = "dbname = 'postgres' host='192.168.1.48' user='postgres' password=''"
connection = db.connect(con)
cur=connection.cursor()

    Create table if not exist in new database

In [48]:
df = pd.read_csv('filtered.csv')
#cur.execute(query='create table if not exists scf(id bigint primary key, summary text, description text, rating int, full_address text, date date, html_url text, zipcode int, coordinate text)')


In [30]:
connection.commit()
cur.close()
connection.close()

    Break down the data and insert into table

In [28]:
df = pd.read_csv('filtered.csv')

In [52]:
for i,r in df.iterrows():
    id = r['id']
    summary = r['summary']
    summary = summary.replace('\'','')
    desc = str(r['description'])
    desc = desc.replace('\'','')
    rating = r['rating']
    f_add = str(r['full_address'])
    f_add = f_add.replace('\'','')
    url = r['html_url']
    dat = r['date']
    zip = r['zip_code']
    if zip == 'nan':
        zip = 95051
    coord = r['coordinate']

    insert = "insert into scf(id, summary, description, rating, full_address, date, html_url, zipcode, coordinate) "
    cur.execute(query=insert + f"values({id},'{summary}','{desc}',{rating},'{f_add}','{dat}','{url}',{zip},'{coord}');")
   

UndefinedColumn: column "nan" does not exist
LINE 1: ...-11-19','https://seeclickfix.com/issues/13718508',nan,'{"lat...
                                                             ^


In [32]:
print(type(dt.datetime.now()))

<class 'datetime.datetime'>
